In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')





from datetime import datetime
from datetime import timedelta
import pandas as pd
import time

In [7]:
def top_line_user(session):
    
    while(True):

        time.sleep(10)

        #take all users
        clients = pd.DataFrame(list(session.execute(f"SELECT * FROM Client;")))

        #for each user get his last index in the event table
        for i in range(clients.shape[0]):

            last_index_Event = pd.DataFrame(list(session.execute(f"SELECT max(id_event) as max FROM Event where \
            id_user = '{clients.loc[i,'id_client']}' ALLOW FILTERING;")))['max'].unique()
            if(last_index_Event[0] != None): #if the user have already used the train
                if(clients.loc[i,'last_index'] == None):
                    clients.loc[i,'last_index'] = -1

                if(clients.loc[i,'last_index'] < last_index_Event):  #if there is new data for the current user then update data        
                        #print(clients.loc[i,'id_client'])
                        client = pd.DataFrame(list(session.execute(f"SELECT * FROM Event where \
                                    id_user = '{clients.loc[i,'id_client']}' ALLOW FILTERING;")))
                        #print(client)
                        most_used_line =list(client.groupby('line').count().reset_index().sort_values(
                            by=['id_card'], ascending=False).head(1)['line'])[0]
                        #print(clients.loc[i,'id_client'])
                        #Update value
                        session.execute(f"UPDATE client SET \
                        top_line ='{most_used_line}' ,last_index ={last_index_Event[0]} WHERE id_client = '{clients.loc[i,'id_client']}';")

                        print(f"top used line for client {clients.loc[i,'id_client']} is updated :)")

In [8]:
if __name__ == "__main__":
    print("Connecting to the Cluster..........")
    #connect to the cluster
    cluster = Cluster(["127.0.0.1"],port=9042)


    #connect to the keyspace
    session = cluster.connect('test')

    print("generating top line statistics.........")

    top_line_user(session)

Connecting to the Cluster..........
generating top line statistics.........
top used line for client 5b7f13b1de5b4b6bac63889649574e9c is updated :)
top used line for client dd098cfb10b345e28529e427b4b97e56 is updated :)
top used line for client 6f705d721b0c4132b8d0fcc87b9e76e8 is updated :)
top used line for client 037de9a0a0064dbd8b2919b6414f5014 is updated :)
top used line for client c19a1973a2e74ac5b1045ecd330b098d is updated :)
top used line for client 06d0dbe474434cd9a398701218de4f2d is updated :)
top used line for client 39337b10a4134452819f592e37b81a74 is updated :)
top used line for client e7fa06ee74cc496bb3a4e83f01a7be51 is updated :)
top used line for client 43c0e89958cd4435b04928c079578658 is updated :)
top used line for client cbe0a4b321ca4225981846a882f326b2 is updated :)
top used line for client 2b0e9f9b67e9455888e66ad3ae36e066 is updated :)
top used line for client ac9fd27f1be540f3ad27ea9214839237 is updated :)
top used line for client 267863f6f17b42d68789e2a2e905a4b6 is

KeyboardInterrupt: 